<a href="https://colab.research.google.com/github/yanqin-denver/Product-Client-Research-Project/blob/master/Product_%26_Research_second_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import file
import pandas as pd
import numpy as np
from google.colab import files
uploaded = files.upload()

Saving Product & Customer Research.xlsx to Product & Customer Research (1).xlsx


In [0]:
import io
df = pd.read_excel(io.BytesIO(uploaded['Product & Customer Research.xlsx']), "Research Results")

In [0]:
df = df.replace(r'^\s*$', np.nan, regex=True)
df = df.dropna()

In [0]:
df = df[(df["Purchase amount during research period"] >= 0) & (df["Historical Sales Volume"] >= 0)]

In [0]:
# drop ID columns 
df = df.drop(["Customer Number"], axis = 1) 
# drop columns of drug that cannot treat Leishmaniasis disease 
df = df.drop(['benznidazole', 'nifurtimox', 'dapsone', 'streptomycin', 'Malarone', 'Ribavirin'], axis = 1)

In [0]:
df['Number of Years Being Customer'] = 2019 - pd.DatetimeIndex(df['Date of First Purchase']).year

In [0]:
# drop date
df = df.drop(["Date of First Purchase"], axis = 1)

In [0]:
# get_dummies
df_X = df.loc[:, df.columns != 'Purchase amount during research period']
features = pd.get_dummies(df_X)

In [0]:
labels = df.loc[:, df.columns == 'Purchase amount during research period']

In [0]:
# Use numpy to convert to arrays
import numpy as np
# Labels are the values we want to predict
labels = np.array(labels)
# Save names of features for later
feature_list = list(features.columns)
# Convert to numpy array
#features = np.array(features)

In [0]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 0)

In [0]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
rfe = RFE(lm,)
rfe = rfe.fit(train_features, train_labels)
print(rfe.support_)
print(rfe.ranking_)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[False False False False False  True  True  True  True  True  True  True
  True  True  True  True  True False False  True  True  True  True False
  True False  True False False False  True  True False False False False
 False  True False  True False False False False False  True False False
 False False False  True  True False  True  True  True  True False False
  True  True  True False False  True]
[34 24 26 28 33  1  1  1  1  1  1  1  1  1  1  1  1 23 27  1  1  1  1 32
  1  7  1 25 13  5  1  1 15  6 30  3 19  1  4  1 20 12 14 11 18  1 22  8
 17 29 10  1  1 16  1  1  1  1 21  2  1  1  1  9 31  1]


In [0]:
train_features = train_features[train_features.columns[rfe.support_]]

In [0]:
train_features.columns

Index(['Repurchase Method_AUTO RENEW', 'Repurchase Method_NOTICE',
       'Last Transaction Channel_AUTO RENEW',
       'Last Transaction Channel_BILLING',
       'Last Transaction Channel_BRANCH (PHONE)',
       'Last Transaction Channel_BRANCH (POS)', 'Last Transaction Channel_IT',
       'Last Transaction Channel_MAIL', 'Last Transaction Channel_PHONE',
       'Last Transaction Channel_WEB', 'Miltefosine_N', 'Miltefosine_Y',
       'Pentamidine_N', 'Pentamidine_Y', 'Ship to country_Algeria',
       'Ship to country_Argentina', 'Ship to country_Bangladesh',
       'Ship to country_Canada', 'Ship to country_France',
       'Ship to country_Germany', 'Ship to country_Jordan',
       'Ship to country_Laos', 'Ship to country_Nigeria',
       'Ship to country_Slovenia', 'Ship to country_South Africa',
       'Ship to country_Spain', 'Ship to country_Sri Lanka',
       'Ship to country_Sudan', 'Ship to country_Taiwan',
       'Ship to country_Turkey', 'Ship to country_USA',
       'Ship to

In [0]:
test_features = test_features[train_features.columns]

In [0]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 0)
# Train the model on training data
rf.fit(train_features, train_labels)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [0]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# predict training set
train_predictions = rf.predict(train_features)
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_labels)))
print("RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(train_labels, train_predictions)))

Mean Absolute Error : 361.7285350361563
RMSE : 702.9


In [0]:
# use xgboost
from xgboost import XGBRegressor
xg_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
xg_model.fit(train_features, train_labels, early_stopping_rounds=5, 
             eval_set=[(test_features, test_labels)], verbose=False)

[10:41:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [0]:
xg_predictions = xg_model.predict(test_features)

In [0]:
# predict training set
train_predictions = xg_model.predict(train_features)
print("Mean Absolute Error : " + str(mean_absolute_error(xg_predictions, test_labels)))
print("RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(train_labels, train_predictions)))

Mean Absolute Error : 356.68184729066604
RMSE : 709.9


In [0]:
# decision tree
from sklearn.tree import DecisionTreeRegressor
DT = DecisionTreeRegressor(max_depth=15, min_samples_leaf=100)
DT.fit(train_features, train_labels)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=15,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=100, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [0]:
# Use the Decision tree's predict method on the test data
predictions = DT.predict(test_features)
# predict training set
train_predictions = DT.predict(train_features)
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_labels)))
print("RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(train_labels, train_predictions)))

Mean Absolute Error : 363.8263410699126
RMSE : 715.3
